#Data cleaning and model prediction

In [ ]:
# The libraries to use are imported

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score



from google.colab import files 
uploaded=files.upload()
import io



Saving SubsidioEmergencia.xlsx to SubsidioEmergencia.xlsx


#DESCRIPTION# 
This notebook performs a deep cleaning of the data set. Additionally, the process carried out with the categorical and numerical data for the selection of a predictive model is also shown that helps to make the decision when assigning the unemployment benefit.

##MAIN FIELDS##

The more important fields found in the data source are the following, here we will explain why we took some decisions on data cleaning or dropping certain fields

**1 Document type**: Represents the type of document used by the users to apply for the subsidies, this field does is mostly registers of the national identity card, therefore, this field will be droped as it is not useful for further modeling or analysis 

**2. Id number**: Representing the id number of those applying for the subsidy, once again this field does not give us valuable information to predict whether someone is going to be elegible for a subsidy or not. Also, due to the right of privacy of the information this field should be discarted

**3. Gender:** This field was filled either masculine or femenine in the documents to apply for the subsidies, in this case we have opted to convert the field in a binary field with one hot encoding, because gender could be a valuable demographic to collect in order to predict wheteher someone is more likely to receive goverment handouts

**5. Civil Status:** We decided to keep this field as sometimes subsidies are more accesible to single moms and dads, thus it may be useful for predicting the likelyhood of receiving the subsidy, we decided to convert this field which was a string into multiple columns with one hot encoding

**6. Municipality:** This field described in which town of Antioquia the applicant lived, however, most of the applicants came from the Medellin Metropolitan region, thus we took the decision to delete the field as it might bias the predictions towards population in cities rather than rural areas 

**7. Ethnicity:** This column represented if the applicant belong to a indigenous etnicity or afro community. We decided to keep this field but we will unify this field in the following fashion: instead of specifying ethnicity we will place a 1 (true if it belongs to any ethnicity) or 0 if they don't. Also we will combine the field with the next field (type of population) which will be described 

**8. Type of population:** This field describe if the applicant belongs to a vulnerable population such as victim of the conflict, ex combatants from the internal conflict. Again, this field will be encoded in a binary fashion, with a 1 representing belonging to a vulnerable group and 0, otherwise. As indicated before, this field and the Ethnicity will be merged following a logical operator OR, thus if its true in at least one case it will be true

**9. Last Salary:** The amount earned in the applicant last salary, we kept the field however, we encoded, giving it a weight, the lower the salary, the greater weight it had. we decided to divide the salary in several categories, starting in 1 with less weight.

**10. Status of the resquest:** This field had several statuses, Analysis Pending, Approved, Denied, Voluntary retirement, Right loss or data not available.  We decided to delete this field, as the vast majority of the data was pending analysis. 

**11. Last type of laboral contract:** This field represented the last type of laboral contract the applicant had, this field was deleted, as the salary has a considerable weight on the decision and it is already an indicator of the wealth of the individual applying for the subsidy.

**12. Previous unemployement subsidy:** In this column we have if the applicant had beeen selected for unemployement subsidies before applying to this one. The column was binarized

**13. In charge of # of persons:** This column had the number of persons the applicant was in charge of, we decided that we would group them in 2 categories, no persons in charge 0, and 1 in the other case

**14. Complies with the requirements for subsidy:** This column indicates if the solicitant complies with the requirements to have access to the subsidy. This field will be the one that we'll try to predict in our model. (The field was changed from Yes and no to 1 and 0) 



In [ ]:
# Loading dataframe from excel file
df = pd.read_excel('SubsidioEmergencia.xlsx', sheet_name='IWvyio')

In [ ]:
#As we will not analyzed duplicate applications we drop duplicate id numbers
df = df.drop_duplicates(['Número de Documento'])

In [ ]:
#Here we drop all the columns that we considered not useful for our analysis, most of them were fields related to paperwork but didn't have any valuable data

df = df.drop(['¿Cuál fue su última Caja de Compensación?','Nombre de la empresa', 'Ciudad/Municipio',
                      'Si en la modalidad de postulación elegiste independiente, por favor selecciona:', '¿A que E.P.S esta afiliado Actualmente?', '¿A que fondo de pensiones está afiliado actualmente?',
                      'Número de la cuenta', 'DECLARO BAJO LA GRAVEDAD DE JURAMENTO QUE:', 'Al postularme como dependiente y no tener certificación de terminación de la relación laboral con mi último empleador, declaro bajo la gravedad de juramento y acepto que ésta',
                      'Autorización protección de datos personales', 'Certificado de terminación de la relación laboral o del contrato', 'Certificado de inscripción al Servicio Público de Empleo',
                      '¿Desea adjuntar otro documento?', '¿Desea adjuntar otro documento?.1','Documentos de la persona a cargo.1', '¿Desea adjuntar otro documento?.2', 'Documentos de la persona a cargo.2',
                      '¿Desea adjuntar otro documento?.3', 'Documentos de la persona a cargo.3', '¿Desea adjuntar otro documento?.4','Documentos de la persona a cargo.4', 'COD. TIPO. DOC',
                      'TIPO.NUM.DOC', 'CUMPLE CARTA', 'ESTADO PAGOS', 'Barrio', 'Términos y condiciones','Documentos de la persona a cargo', '¿Ha cambiado el número de personas a cargo con respecto a su condición de trabajador?','Start Date (UTC)', 'Submit Date (UTC)', 'Network ID',
                     'Start Date (UTC).1', 'Submit Date (UTC).1', 'HORAS DIF.', 'Start Date (UTC) COLOMBIA', 'Submit Date (UTC) COLOMBIA', 'LOTE - DÍA','Tipo de Documento','Número de Documento','Municipio','¿Trabaja actualmente?','¿Está inscrito en el servicio público de empleo?','Fecha de nacimiento',
                      'Departamento','Fecha de terminación laboral','Departamento.1','Modalidad de Postulación','¿Recibió este subsidio en los últimos 3 años?','¿Recibía Cuota Monetaria por las personas a cargo?','Tipo de cuenta:', 'Entidad', 'REGION', 'ORIGEN DE POSTULACIÓN',
       'TIPO DOCUMENTO ERRADO\nIngresado por Usuario','NÚMERO DOCUMENTO ERRADO\nIngresado por Usuario', 'CUMPLE INSCRIPCIÒN SISE','USUARIO DESISTE O PIERDE EL BENEFICIO','ESTADO TABLERO','ESTADO BASE IVR','Factor de vulnerabilidad'], axis=1)

In [ ]:
#Displays columns after the first column filter

df.columns

Index(['Género', 'Estado civil', 'Zona', 'Pertenencia étnica', 'Población',
       'Último salario', 'Tipo de vinculación laboral',
       '¿Ha recibido subsidio al desempleo?',
       '¿Por cuantas personas a cargo recibía cuota monetaria?',
       'CUMPLE REQUISITOS DE SUBSIDIO'],
      dtype='object')

In [ ]:
#The civil status category, before we start with the one hot encoding, the fields were unified under several categories

df['Estado civil'] = df['Estado civil'].fillna('No disponible')

df[df['Estado civil'].str.contains("Soltero/a", case=False)]
df['Estado civil'] = df['Estado civil'].apply(lambda x: x.replace('Soltero/a', 'Soltero'))

df[df['Estado civil'].str.contains("Casado/a", case=False)]
df['Estado civil'] = df['Estado civil'].apply(lambda x: x.replace('Casado/a', 'Casado'))

df[df['Estado civil'].str.contains("Unión Libre", case=False)]
df['Estado civil'] = df['Estado civil'].apply(lambda x: x.replace('Unión Libre', 'Unión libre'))

df[df['Estado civil'].str.contains("Separado/a", case=False)]
df['Estado civil'] = df['Estado civil'].apply(lambda x: x.replace('Separado/a', 'Separado'))

df[df['Estado civil'].str.contains("Viudo/a", case=False)]
df['Estado civil'] = df['Estado civil'].apply(lambda x: x.replace('Viudo/a', 'Viudo'))

In [ ]:

# A count is made by marital status to verify previous cleaning
count_estado_civil = df.groupby(df['Estado civil'])['Población'].count().sort_values(ascending=False)
count_estado_civil

Estado civil
Soltero          36369
Unión libre      22120
Casado           16207
No disponible     6917
Separado          4430
Viudo              511
Name: Población, dtype: int64

In [ ]:
#Here the population was changed from several sub categories to vulerable population or does not apply to vulnerable population status

df[df['Población'].str.contains('Victima del conflicto armado y en condiciones de desplazamiento', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('Victima del conflicto armado y en condiciones de desplazamiento', 'poblacion_vulnerable'))

df[df['Población'].str.contains('Victima del conflicto armado y en condiciones de discapacidad', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('Victima del conflicto armado y en condiciones de discapacidad', 'poblacion_vulnerable'))

df[df['Población'].str.contains('Victima del conflicto armado y en condiciones de desplazamiento', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('Victima del conflicto armado y en condiciones de discapacidad', 'poblacion_vulnerable'))

df[df['Población'].str.contains('Victima del conflicto armado y en condiciones de desplazamiento', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('Victima del conflicto armado y en condiciones de discapacidad', 'poblacion_vulnerable'))

df[df['Población'].str.contains('Victima del conflicto armado y en condiciones de desplazamiento', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('No disponible', 'No aplica'))

df[df['Población'].str.contains('Victima del conflicto armado y en condiciones de desplazamiento', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('No disponible', 'No aplica'))

df[df['Población'].str.contains('En condición de desplazamiento', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('En condición de desplazamiento', 'poblacion_vulnerable'))

df[df['Población'].str.contains('Victima del conflicto armado', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('Victima del conflicto armado', 'poblacion_vulnerable'))

df[df['Población'].str.contains('Victima del conflicto armado', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('Victima del conflicto armado', 'poblacion_vulnerable'))

df[df['Población'].str.contains('En condición de discapacidad física', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('En condición de discapacidad física', 'poblacion_vulnerable'))


df[df['Población'].str.contains('En condiciones de desplazamiento y en condiciones de discapacidad física', case=False)]
df['Población'] = df['Población'].apply(lambda x: x.replace('En condiciones de desplazamiento y en condiciones de discapacidad física', 'poblacion_vulnerable'))


In [ ]:
#A count is made by population state to verify previous cleaning
count_vulnerabilidad = df.groupby(df['Población'])['Estado civil'].count().sort_values(ascending=False)
count_vulnerabilidad

Población
No aplica               76153
poblacion_vulnerable    10401
Name: Estado civil, dtype: int64

In [ ]:
#Here the registers from Ethnic populations were changed to Belongs or does not belong to an ethnic group

df[df['Pertenencia étnica'].str.contains('En condiciones de desplazamiento y en condiciones de discapacidad física', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('No se reconoce en ninguno de los anteriores', 'no pertenece'))

df[df['Pertenencia étnica'].str.contains('No disponible', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('No disponible', 'no pertenece'))


df[df['Pertenencia étnica'].str.contains('Afrocolombiano', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('Afrocolombiano', 'si pertenece'))

df[df['Pertenencia étnica'].str.contains('Indigena ', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('Indigena', 'si pertenece'))

df[df['Pertenencia étnica'].str.contains('Comunidad negra', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('Comunidad negra', 'si pertenece'))

df[df['Pertenencia étnica'].str.contains('Palanquero', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('Palanquero', 'si pertenece'))

df[df['Pertenencia étnica'].str.contains('Raizal Archipielago de San Andrés, Providencia y Santa Catalina', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('Raizal Archipielago de San Andrés, Providencia y Santa Catalina', 'si pertenece'))

df[df['Pertenencia étnica'].str.contains('Room / Gitano', case=False)]
df['Pertenencia étnica'] = df['Pertenencia étnica'].apply(lambda x: x.replace('Room / Gitano', 'si pertenece'))






In [ ]:
#A count is made by ethnicity to verify previous cleaning
count_etnia = df.groupby(df['Pertenencia étnica'])['Estado civil'].count().sort_values(ascending=False)
count_etnia

Pertenencia étnica
no pertenece    78411
si pertenece     8143
Name: Estado civil, dtype: int64

In [ ]:
#The last salary was separated into 4 separate categories from 1-2 minimum wages per month, to more than 4 minimum wages per month
df['Último salario'] = df['Último salario'].fillna('No disponible')

df['Último salario'] = df['Último salario'].replace(['Desde 1 hasta 2 smmlv', 'Desde 1 hasta 2 SMMLV (Desde $980.657 hasta $1.961.314)'], '1-2 smmlv')

df['Último salario'] = df['Último salario'].replace(['Mayor a  3 hasta 4 smmlv', 'Mayor a 3 hasta 4 SMMLV (Mayor a $2.941.971 hasta $3.922.628)'], '3-4 smmlv')

df['Último salario'] = df['Último salario'].replace(['Mayor a 4 smmlv','Mayor a 4 SMMLV (Mayor a $3.922.628)'], '>4 smmlv')

df[df['Último salario'].str.contains('Mayor a 2 hasta 3 smmlv', case=False)]
df['Último salario'] = df['Último salario'].apply(lambda x: x.replace('Mayor a 2 hasta 3 smmlv', '2-3 smmlv'))


df[df['Último salario'].str.contains('Mayor a 2 hasta 3 SMMLV (Mayor a $1.961.314 hasta $2.941.971)', case=False)]
df['Último salario'] = df['Último salario'].apply(lambda x: x.replace('Mayor a 2 hasta 3 SMMLV (Mayor a $1.961.314 hasta $2.941.971)', '2-3 smmlv'))


/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [ ]:
#A count is made by the salary interval to verify previous cleaning
count_salario = df.groupby(df['Último salario'])['Estado civil'].count().sort_values(ascending=False)
count_salario

Último salario
1-2 smmlv        65741
>4 smmlv         15588
2-3 smmlv         4004
3-4 smmlv         1148
No disponible       73
Name: Estado civil, dtype: int64

In [ ]:
#In this field, a lot of empty spaces are erased, and it is only encoded as yes or no

df['¿Ha recibido subsidio al desempleo?'] = df['¿Ha recibido subsidio al desempleo?'].replace(['No                                  '], 'No')
df['¿Ha recibido subsidio al desempleo?'] = df['¿Ha recibido subsidio al desempleo?'].replace(['Si                                  '], 'Si')



In [ ]:
##A count is made by have previously received the grant to verify previous cleaning

count_sub_prev = df.groupby(df['¿Ha recibido subsidio al desempleo?'])['Estado civil'].count().sort_values(ascending=False)
count_sub_prev

¿Ha recibido subsidio al desempleo?
No    81784
Si     4697
Name: Estado civil, dtype: int64

In [ ]:
#Here the persons in charge field was changed from a number of persons the applicant was in charge of to yes(Si) if he/she was resposible for someone else or No in the other case 
df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].fillna('No disponible')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([1], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([2], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([3], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([0], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([4], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([5], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([6], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([7], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([8], 'Si')

df['¿Por cuantas personas a cargo recibía cuota monetaria?'] = df['¿Por cuantas personas a cargo recibía cuota monetaria?'].replace([9], 'Si')


In [ ]:
#A count is made to verify previous cleaning
count_sub_prev = df.groupby(df['¿Por cuantas personas a cargo recibía cuota monetaria?'])['Estado civil'].count().sort_values(ascending=False)
count_sub_prev

¿Por cuantas personas a cargo recibía cuota monetaria?
No disponible                   48593
Si                              36327
                                 1634
Name: Estado civil, dtype: int64

In [ ]:
#This will be the variable that we will try to predict
df['CUMPLE REQUISITOS DE SUBSIDIO'] = df['CUMPLE REQUISITOS DE SUBSIDIO'].str.lower()
df['CUMPLE REQUISITOS DE SUBSIDIO'] = df['CUMPLE REQUISITOS DE SUBSIDIO'].fillna('No disponible')

In [ ]:
#A count of the variable to be predicted is performed
count_sub_prev = df.groupby(df['CUMPLE REQUISITOS DE SUBSIDIO'])['Estado civil'].count().sort_values(ascending=False)
count_sub_prev

CUMPLE REQUISITOS DE SUBSIDIO
si cumple        61707
No disponible    13240
no cumple        11607
Name: Estado civil, dtype: int64

In [ ]:
#A count of the variable to be predicted is performed, everything is unified in lowercase
df['CUMPLE REQUISITOS DE SUBSIDIO'] = df['CUMPLE REQUISITOS DE SUBSIDIO'].str.lower()
df['CUMPLE REQUISITOS DE SUBSIDIO'] = df['CUMPLE REQUISITOS DE SUBSIDIO'].fillna('No disponible')
count_sub_prev = df.groupby(df['CUMPLE REQUISITOS DE SUBSIDIO'])['Estado civil'].count().sort_values(ascending=False)
count_sub_prev

CUMPLE REQUISITOS DE SUBSIDIO
si cumple        61707
no disponible    13240
no cumple        11607
Name: Estado civil, dtype: int64

In [ ]:
#The gender count is performed, this variable is not modified

count_genero = df.groupby(df['Género'])['Estado civil'].count().sort_values(ascending=False)
count_genero

Género
Masculino    45389
Femenino     41165
Name: Estado civil, dtype: int64

In [ ]:
#Counting by Zone is performed, this variable is not modified
count_zona = df.groupby(df['Zona'])['Estado civil'].count().sort_values(ascending=False)
count_zona

Zona
Urbana           71056
Rural            10097
No disponible     5401
Name: Estado civil, dtype: int64

In [ ]:
#Cleaning according to the type of labor bonding The NANs are modified by "Not available"
df["Tipo de vinculación laboral"] = df["Tipo de vinculación laboral"].fillna('No disponible')



In [ ]:
#A count is made according to the type of employment relationship to verify the previous cleaning
count_tipo_vinculacion = df.groupby(df['Tipo de vinculación laboral'])['Estado civil'].count().sort_values(ascending=False)
count_tipo_vinculacion

Tipo de vinculación laboral
Indefinido                 27741
Termino fijo               20369
Obra o labor contratada    18064
No disponible              15848
Prestación de Servicios     4532
Name: Estado civil, dtype: int64

In [ ]:
#The new dimension is verified after the entire cleaning process
df.shape

(86554, 10)

#Encoding process

As they are categorical and numerical variables, it is necessary to make a coding which allows their compatibility with the different classification and prediction models

In [ ]:
#A copy of the original data frame is created to start the coding process to make the model
df2=df.copy()

In [ ]:
# In this first process, the variables are taken and codified with get_dummies, to assign columns with 0 and 1 as they have a characteristic

cat_vars=['Género','Zona','Estado civil','Pertenencia étnica','Población','¿Por cuantas personas a cargo recibía cuota monetaria?']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df2[var], prefix=var)
    data1=df2.join(cat_list)
    df2=data1
cat_vars=['Género','Zona','Estado civil','Pertenencia étnica','Población','¿Por cuantas personas a cargo recibía cuota monetaria?']
data_vars=df2.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [ ]:
# Encode ordinal variables

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder
X = {'No disponible':1, '>4 smmlv':2,'3-4 smmlv':3,'2-3 smmlv':4,'1-2 smmlv':5}
df2['Último salario']=df2['Último salario'].map(X)
Y = {'No disponible':1,'Prestación de Servicios':2, 'Obra o labor contratada':3,'Termino fijo':4,'Indefinido':5}
df2['Tipo de vinculación laboral']=df2['Tipo de vinculación laboral'].map(Y)

from sklearn import preprocessing
labelencoder= preprocessing.LabelEncoder()
df2["¿Ha recibido subsidio al desempleo?"]=df2["¿Ha recibido subsidio al desempleo?"].apply(lambda x:0 if x=="No" else 1)
df2["CUMPLE REQUISITOS DE SUBSIDIO"]=df2["CUMPLE REQUISITOS DE SUBSIDIO"].apply(lambda x:0 if (x=="no cumple"or x=="no disponible") else 1)
s = df2.pop('CUMPLE REQUISITOS DE SUBSIDIO')
df2 = pd.concat([df2, s], 1)
print(df2)
df2

          Género  ... CUMPLE REQUISITOS DE SUBSIDIO
0       Femenino  ...                             0
1       Femenino  ...                             0
2       Femenino  ...                             0
3       Femenino  ...                             0
4       Femenino  ...                             1
...          ...  ...                           ...
96749  Masculino  ...                             0
96750  Masculino  ...                             0
96751  Masculino  ...                             0
96754   Femenino  ...                             0
96755   Femenino  ...                             0

[86554 rows x 28 columns]


,Género,Estado civil,Zona,Pertenencia étnica,Población,Último salario,Tipo de vinculación laboral,¿Ha recibido subsidio al desempleo?,¿Por cuantas personas a cargo recibía cuota monetaria?,Género_Femenino,Género_Masculino,Zona_No disponible,Zona_Rural,Zona_Urbana,Estado civil_Casado,Estado civil_No disponible,Estado civil_Separado,Estado civil_Soltero,Estado civil_Unión libre,Estado civil_Viudo,Pertenencia étnica_no pertenece,Pertenencia étnica_si pertenece,Población_No aplica,Población_poblacion_vulnerable,¿Por cuantas personas a cargo recibía cuota monetaria?_,¿Por cuantas personas a cargo recibía cuota monetaria?_No disponible,¿Por cuantas personas a cargo recibía cuota monetaria?_Si,CUMPLE REQUISITOS DE SUBSIDIO
0,Femenino,Soltero,No disponible,no pertenece,No aplica,5,1,0,Si,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
1,Femenino,Soltero,No disponible,no pertenece,No aplica,5,1,0,Si,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
2,Femenino,Casado,No disponible,no pertenece,No aplica,5,1,0,Si,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0
3,Femenino,Soltero,No disponible,no pertenece,No aplica,5,1,0,Si,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
4,Femenino,Soltero,No disponible,no pertenece,No aplica,5,1,0,Si,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96749,Masculino,Soltero,Urbana,si pertenece,No aplica,5,4,0,No disponible,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0
96750,Masculino,Soltero,Urbana,no pertenece,No aplica,4,4,0,No disponible,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0
96751,Masculino,Unión libre,Urbana,no pertenece,poblacion_vulnerable,5,3,1,No disponible,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0
96754,Femenino,Unión libre,Urbana,no pertenece,No aplica,3,5,0,No disponible,1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0


In [ ]:
#The previously coded columns are deleted so that only a matrix with 0 and 1 remains

df2.drop(columns=['Género','Zona','Estado civil','Pertenencia étnica','Población','¿Por cuantas personas a cargo recibía cuota monetaria?','Tipo de vinculación laboral','Último salario'],inplace=True)

In [ ]:
# it is verified that the new data only contains 0 and 1
df2.head()

,¿Ha recibido subsidio al desempleo?,Género_Femenino,Género_Masculino,Zona_No disponible,Zona_Rural,Zona_Urbana,Estado civil_Casado,Estado civil_No disponible,Estado civil_Separado,Estado civil_Soltero,Estado civil_Unión libre,Estado civil_Viudo,Pertenencia étnica_no pertenece,Pertenencia étnica_si pertenece,Población_No aplica,Población_poblacion_vulnerable,¿Por cuantas personas a cargo recibía cuota monetaria?_,¿Por cuantas personas a cargo recibía cuota monetaria?_No disponible,¿Por cuantas personas a cargo recibía cuota monetaria?_Si,CUMPLE REQUISITOS DE SUBSIDIO
0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
2,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0
3,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
4,0,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,1


In [ ]:
#The variable to be predicted is counted again
count_sub_prev = df2.groupby(df2['CUMPLE REQUISITOS DE SUBSIDIO'])['Género_Masculino'].count().sort_values(ascending=False)
count_sub_prev

CUMPLE REQUISITOS DE SUBSIDIO
1    61707
0    24847
Name: Género_Masculino, dtype: int64

In [ ]:
#Coded columns are listed
df2.columns

Index(['¿Ha recibido subsidio al desempleo?', 'Género_Femenino',
       'Género_Masculino', 'Zona_No disponible', 'Zona_Rural', 'Zona_Urbana',
       'Estado civil_Casado', 'Estado civil_No disponible',
       'Estado civil_Separado', 'Estado civil_Soltero',
       'Estado civil_Unión libre', 'Estado civil_Viudo',
       'Pertenencia étnica_no pertenece', 'Pertenencia étnica_si pertenece',
       'Población_No aplica', 'Población_poblacion_vulnerable',
       '¿Por cuantas personas a cargo recibía cuota monetaria?_                            ',
       '¿Por cuantas personas a cargo recibía cuota monetaria?_No disponible',
       '¿Por cuantas personas a cargo recibía cuota monetaria?_Si',
       'CUMPLE REQUISITOS DE SUBSIDIO'],
      dtype='object')

In [ ]:
#split data set in test and train. 
y=df2['CUMPLE REQUISITOS DE SUBSIDIO']
X=df2.iloc[:,:-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#create logisticRegression model , fit model with train data and use some metrics to know how will be model performance
#Logistic regression is a classification algorithm, used when the value of the target variable is categorical in nature. Logistic regression is most commonly used when the data in question has binary output, so when it belongs to one class or another, or is either a 0 or 1
model = LogisticRegression()
model = model.fit(X_train,y_train)
predicted = model.predict(X_test)
print(metrics.confusion_matrix(y_test, predicted))
scores = cross_val_score(model, X_train, y_train, scoring="f1")
print(scores)
scores= scores[np.logical_not(np.isnan(scores))]
print("Loss: {0:.3f} (+/- {1:.3f})".format(scores.mean(), scores.std()))

[[ 4119  4013]
 [ 1219 19212]]
[0.88033463 0.87844867 0.88048394 0.88006345 0.88324643]
Loss: 0.881 (+/- 0.002)


In [ ]:
#use F1 Score to know model performe
from sklearn.metrics import f1_score
y_train_pred = model.predict(X_train)
score = f1_score(y_train, y_train_pred)
print(f"F1 Score: {score}")
y_test_pred = model.predict(X_test)
score = f1_score(y_test, y_test_pred)
print(f"F1 Score: {score}")

F1 Score: 0.880540827746815
F1 Score: 0.880153930731171


In [ ]:
# The prediction is verified for the first 10 examples, 
#in the first column it refers to the real value and the
#second the prediction value given by the model
y_pred = pd.Series(model.predict(X_test))
y_test = y_test.reset_index(drop=True)
z = pd.concat([y_test, y_pred], axis=1)
z.columns = ['original label', 'Model prediction']
z.head(10)

,original label,Model prediction
0,1,1
1,1,1
2,0,0
3,0,1
4,1,1
5,0,0
6,1,1
7,0,0
8,1,1
9,0,1


In [ ]:
#the second model will be RandomForest wit n_estimators=1000. 
#A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree.

model1 = Pipeline(
    [
        ('ran', RandomForestClassifier(n_estimators=1000))
    ]
)
model1.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('ran',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=1000, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [ ]:
# Add more parameters to RandomForest model to improve performance
param_grid = {
    "ran__n_estimators": [10,100,1000],
    "ran__max_depth": [10, 50, 100],
    "ran__max_features": ["auto", "log2"]
}

gs = GridSearchCV(
    model1,
    param_grid,
    scoring="f1",
    return_train_score=True
).fit(X_train, y_train)


In [ ]:
# Probability to predict each class . 
probability = model.predict_proba(X_test)
print (probability)

[[0.13490733 0.86509267]
 [0.10054158 0.89945842]
 [0.03853125 0.96146875]
 ...
 [0.13490733 0.86509267]
 [0.13376096 0.86623904]
 [0.12059105 0.87940895]]


In [ ]:
#predict with Random Forest model and use F1 score to know how is their performance
y_train_pred = model1.predict(X_train)
score = f1_score(y_train, y_train_pred)
print(f"F1 Score: {score}")
y_test_pred = model1.predict(X_test)
score = f1_score(y_test, y_test_pred)
print(f"F1 Score: {score}")

F1 Score: 0.9301004476881983
F1 Score: 0.92802356496624


In [ ]:
#Use GradientBoostingClassifier model and use metrics to calculate their performance 
#GB builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions. In each stage n_classes_ regression trees are fit on the negative gradient of the binomial or multinomial deviance loss function. Binary classification is a special case where only a single regression tree is induced
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
clf.predict(X_test)
clf.score(X_test, y_test)

0.8675559289990548

We use Random Forest, Logistic Regression and GradientBoosting Classifier because all of them have good performance and they are very useful on classication models. Random forest and Logistic Regretion has better F1 score than GradientBoostingClassifier . F1 score is a measure of a test's accuracy. It is calculated from the precision and recall of the test and it is very accurate on the result.

> Bloc con sangría

